In [1]:
import time

import numpy as np
import pandas as pd
import psutil
import torch
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv, GCNConv, SAGEConv
from torch_geometric.utils import to_undirected
from ucimlrepo import fetch_ucirepo

try:
    import pynvml
    pynvml.nvmlInit()
    NVML_AVAILABLE = True
except:
    NVML_AVAILABLE = False

/home/klema/miniconda3/envs/graphsage/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CSV_PATH = '/home/klema/sibnn/gnn_tbank/check_notebooks/data/accepted_2007_to_2018Q4.csv'

In [ ]:
print("Загрузка данных (эмбеддинги для категориальных признаков, граф — только на числовых)")

usecols = [
    'loan_amnt', 'int_rate', 'installment', 'grade', 'emp_length', 
    'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
    'purpose', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc',
    'pub_rec', 'revol_bal', 'revol_util', 'total_acc'
]

df = pd.read_csv(CSV_PATH, usecols=usecols, low_memory=False)

# Фильтрация завершённых займов
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]
y = df['loan_status'].map({'Fully Paid': 0, 'Charged Off': 1}).values

# Числовые признаки
numeric_features = ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
                   'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 
                   'revol_bal', 'revol_util', 'total_acc']
X_num = df[numeric_features].copy()
X_num.replace([np.inf, -np.inf], np.nan, inplace=True)
X_num.fillna(X_num.median(), inplace=True)

# Стандартизация числовых признаков (ТОЛЬКО на них строим граф!)
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)

# Категориальные признаки -> индексы для эмбеддингов
categorical_features = ['grade', 'emp_length', 'home_ownership', 'verification_status', 'purpose']
X_cat = df[categorical_features].copy()
X_cat.fillna('Unknown', inplace=True)

# Label Encoding для каждого категориального признака
X_cat_encoded = np.zeros((len(df), len(categorical_features)), dtype=np.int64)
cat_dims = []

for i, col in enumerate(categorical_features):
    le = LabelEncoder()
    X_cat_encoded[:, i] = le.fit_transform(X_cat[col].astype(str))
    cat_dims.append(len(le.classes_))
    print(f"  {col}: {len(le.classes_)} уникальных значений")

k = 10
adj_matrix = kneighbors_graph(X_num_scaled, k, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(adj_matrix.nonzero()), dtype=torch.long)
edge_index = to_undirected(edge_index)

# Разделение на train/val/test с сохранением баланса классов
num_nodes = len(df)
indices = np.arange(num_nodes)
train_idx, temp_idx = train_test_split(indices, test_size=0.4, random_state=42, stratify=y)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42, stratify=y[temp_idx])

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[train_idx] = True
val_mask[val_idx] = True
test_mask[test_idx] = True

# Создание объекта данных с разделенными признаками
data = Data(
    x_num=torch.tensor(X_num_scaled, dtype=torch.float),
    x_cat=torch.tensor(X_cat_encoded, dtype=torch.long),
    edge_index=edge_index,
    y=torch.tensor(y, dtype=torch.long),
    train_mask=train_mask,
    val_mask=val_mask,
    test_mask=test_mask
)

# Метаданные для моделей
data.num_numerical = X_num_scaled.shape[1]
data.num_categorical = len(categorical_features)
data.cat_dims = cat_dims

print(f"\nЗагружено {num_nodes} узлов")
print(f"Числовые признаки: {data.num_numerical}")
print(f"Категориальные признаки: {data.num_categorical} ({categorical_features})")
print(f"Уникальные значения по кат. признакам: {cat_dims}")
print(f"Train/Val/Test: {train_mask.sum().item()}/{val_mask.sum().item()}/{test_mask.sum().item()}")
print(f"Рёбер: {edge_index.shape[1]} (k={k})")

class_counts = np.bincount(y)
print(f"Распределение классов: Fully Paid (0): {class_counts[0]}, Charged Off (1): {class_counts[1]}")
print(f"Доля дефолтов: {class_counts[1] / num_nodes:.2%}")

Загрузка данных (эмбеддинги для категориальных признаков, граф — только на числовых)
  grade: 7 уникальных значений
  emp_length: 12 уникальных значений
  home_ownership: 6 уникальных значений
  verification_status: 3 уникальных значений
  purpose: 14 уникальных значений


In [ ]:
# Вычисление весов перед обучением
y_train = data.y[data.train_mask].cpu().numpy()
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

# Используем устройство целевой переменной (всегда присутствует)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(data.y.device)

# Использование в лоссе
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
def compute_metrics(pred, true, probs=None):
    pred = pred.cpu().numpy()
    true = true.cpu().numpy()
    
    metrics = {
        'accuracy': (pred == true).mean(),
        'precision': precision_score(true, pred, zero_division=0),
        'recall': recall_score(true, pred, zero_division=0),
        'f1': f1_score(true, pred, zero_division=0),
    }
    
    if probs is not None:
        probs = probs.cpu().numpy()
        metrics['roc_auc'] = roc_auc_score(true, probs)
        metrics['pr_auc'] = average_precision_score(true, probs)
    
    return metrics

def accuracy(pred_y, y):
    return ((pred_y == y).sum() / len(y)).item()

def test(model, data):
    """Тестирование с поддержкой обеих сигнатур: (x, edge_index) и (x_num, x_cat, edge_index)"""
    model.eval()
    with torch.no_grad():
        # Автоматическое определение сигнатуры модели по количеству параметров в forward
        if hasattr(data, 'x_num') and hasattr(data, 'x_cat'):
            _, out = model(data.x_num, data.x_cat, data.edge_index)
        else:
            _, out = model(data.x, data.edge_index)
        
        pred = out.argmax(dim=1)[data.test_mask]
        probs = torch.exp(out)[:, 1][data.test_mask]  # Вероятность класса 1 (дефолт)
        true = data.y[data.test_mask]
        
        return compute_metrics(pred, true, probs)

In [ ]:
import torch.nn as nn

class FeatureEncoder(torch.nn.Module):
    def __init__(self, cat_dims, cat_emb_dims, num_numerical, output_dim):
        super().__init__()
        assert len(cat_dims) == len(cat_emb_dims)
        
        self.embeddings = torch.nn.ModuleList([
            torch.nn.Embedding(num_categories, emb_dim)
            for num_categories, emb_dim in zip(cat_dims, cat_emb_dims)
        ])
        
        total_input_dim = sum(cat_emb_dims) + num_numerical
        self.projector = torch.nn.Sequential(
            torch.nn.Linear(total_input_dim, output_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )
    
    def forward(self, x_num, x_cat):
        embeddings = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
        x = torch.cat([x_num] + embeddings, dim=1)
        return self.projector(x)

In [ ]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, cat_dims, cat_emb_dims, num_numerical, dim_h, dim_out):
        super().__init__()
        # Энкодер признаков: эмбеддинги + проекция
        self.feature_encoder = FeatureEncoder(
            cat_dims=cat_dims,
            cat_emb_dims=cat_emb_dims,
            num_numerical=num_numerical,
            output_dim=dim_h # проецируем сразу в размерность первого GCN-слоя
        )
        self.sage1 = SAGEConv(dim_h, dim_h)
        self.sage2 = SAGEConv(dim_h, dim_out)
        
        # Оптимизатор с дифференцированной регуляризацией
        self.optimizer = torch.optim.Adam([
            {'params': self.feature_encoder.embeddings.parameters(), 'weight_decay': 1e-3},
            {'params': self.feature_encoder.projector.parameters(), 'weight_decay': 5e-4},
            {'params': self.gcn1.parameters(), 'weight_decay': 5e-4},
            {'params': self.gcn2.parameters(), 'weight_decay': 5e-4},
        ], lr=0.02)

    def forward(self, x_num, x_cat, edge_index):
        x = self.feature_encoder(x_num, x_cat)
        h = self.sage1(x, edge_index)
        h = torch.relu(h)
        h = F.dropout(h, p=0.3, training=self.training)
        h = self.sage2(h, edge_index)
        return h, F.log_softmax(h, dim=1)

    def fit(self, data, epochs, patience=20):
        # Взвешенный лосс для дисбаланса классов
        y_train = data.y[data.train_mask].cpu().numpy()
        weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        weights = torch.tensor(weights, dtype=torch.float).to(data.x_num.device)
        criterion = torch.nn.CrossEntropyLoss(weight=weights)
        optimizer = self.optimizer

        best_val_loss = float('inf')
        patience_counter = 0
        best_state = None

        self.train()
        for epoch in range(epochs + 1):
            optimizer.zero_grad()
            _, out = self(data.x_num, data.x_cat, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                train_pred = out[data.train_mask].argmax(dim=1)
                train_metrics = compute_metrics(train_pred, data.y[data.train_mask])
                
                val_pred = out[data.val_mask].argmax(dim=1)
                val_probs = torch.exp(out)[:, 1][data.val_mask]
                val_metrics = compute_metrics(val_pred, data.y[data.val_mask], probs=val_probs)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            # Ранняя остановка
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                best_state = {k: v.cpu() for k, v in self.state_dict().items()}
            else:
                patience_counter += 1
                if patience_counter >= patience and epoch > 30:
                    print(f"\nEarly stopping at epoch {epoch} (best val loss: {best_val_loss:.3f})")
                    self.load_state_dict(best_state)
                    break

            if epoch % 10 == 0:
                print(f'Epoch {epoch:>3} | '
                    f'TL: {loss:.3f} | TF1: {train_metrics["f1"]:.3f} | '
                    f'VL: {val_loss:.3f} | VF1: {val_metrics["f1"]:.3f} | '
                    f'VRec: {val_metrics["recall"]:.3f} | VPR: {val_metrics["pr_auc"]:.3f}')

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, cat_dims, cat_emb_dims, num_numerical, dim_h, dim_out):
        super().__init__()
        # Энкодер признаков: эмбеддинги + проекция
        self.feature_encoder = FeatureEncoder(
            cat_dims=cat_dims,
            cat_emb_dims=cat_emb_dims,
            num_numerical=num_numerical,
            output_dim=dim_h  # проецируем сразу в размерность первого GCN-слоя
        )
        
        self.gcn1 = GCNConv(dim_h, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_out)
        
        # Оптимизатор с дифференцированной регуляризацией
        self.optimizer = torch.optim.Adam([
            {'params': self.feature_encoder.embeddings.parameters(), 'weight_decay': 1e-3},
            {'params': self.feature_encoder.projector.parameters(), 'weight_decay': 5e-4},
            {'params': self.gcn1.parameters(), 'weight_decay': 5e-4},
            {'params': self.gcn2.parameters(), 'weight_decay': 5e-4},
        ], lr=0.02)

    def forward(self, x_num, x_cat, edge_index):
        # Кодирование признаков через эмбеддинги
        x = self.feature_encoder(x_num, x_cat)
        
        h = F.dropout(x, p=0.3, training=self.training)
        h = self.gcn1(h, edge_index)
        h = torch.relu(h)
        h = F.dropout(h, p=0.3, training=self.training)
        h = self.gcn2(h, edge_index)
        return h, F.log_softmax(h, dim=1)

    def fit(self, data, epochs, patience=20):
        # Взвешенный лосс для дисбаланса классов
        y_train = data.y[data.train_mask].cpu().numpy()
        weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        weights = torch.tensor(weights, dtype=torch.float).to(data.x_num.device)
        criterion = torch.nn.CrossEntropyLoss(weight=weights)
        optimizer = self.optimizer

        best_val_loss = float('inf')
        patience_counter = 0
        best_state = None

        self.train()
        for epoch in range(epochs + 1):
            optimizer.zero_grad()
            _, out = self(data.x_num, data.x_cat, data.edge_index)  # ← ключевое изменение
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                # Метрики на трейне
                train_pred = out[data.train_mask].argmax(dim=1)
                train_metrics = compute_metrics(train_pred, data.y[data.train_mask])
                
                # Метрики на валидации
                val_pred = out[data.val_mask].argmax(dim=1)
                val_probs = torch.exp(out)[:, 1][data.val_mask]
                val_metrics = compute_metrics(val_pred, data.y[data.val_mask], probs=val_probs)
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])

            # Ранняя остановка
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                best_state = {k: v.cpu() for k, v in self.state_dict().items()}
            else:
                patience_counter += 1
                if patience_counter >= patience and epoch > 30:
                    print(f"\nEarly stopping at epoch {epoch} (best val loss: {best_val_loss:.3f})")
                    self.load_state_dict(best_state)
                    break

            if epoch % 10 == 0:
                print(f'Epoch {epoch:>3} | '
                    f'TL: {loss:.3f} | TF1: {train_metrics["f1"]:.3f} | '
                    f'VL: {val_loss:.3f} | VF1: {val_metrics["f1"]:.3f} | '
                    f'VRec: {val_metrics["recall"]:.3f} | VPR: {val_metrics["pr_auc"]:.3f}')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

def monitor_resources():
    stats = {}
    # CPU & RAM
    stats['ram_mb'] = psutil.virtual_memory().used / (1024 ** 2)
    stats['cpu_percent'] = psutil.cpu_percent()

    # GPU
    if device.type == 'cuda' and NVML_AVAILABLE:
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        util = pynvml.nvmlDeviceGetUtilizationRates(handle)
        stats['gpu_mem_mb'] = mem_info.used / (1024 ** 2)
        stats['gpu_util'] = util.gpu
    else:
        stats['gpu_mem_mb'] = None
        stats['gpu_util'] = None
    return stats

def train_with_monitoring(model, data, epochs, model_name):
    print(f"\nTraining {model_name} with resource monitoring\n")
    
    if device.type == 'cuda':
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.empty_cache()

    start_time = time.time()

    model.fit(data, epochs)

    final_ram = psutil.virtual_memory().used / (1024 ** 2)
    max_gpu_mem = None
    if device.type == 'cuda':
        max_gpu_mem = torch.cuda.max_memory_allocated() / (1024 ** 2)

    duration = time.time() - start_time
    test_metrics = test(model, data)

    results = {
        'test_metrics': test_metrics,
        'training_time_sec': duration,
        'final_ram_mb': final_ram,
        'max_gpu_mem_mb': max_gpu_mem,
    }

    print(f"\n{model_name} finished")
    print(f"Test Recall: {test_metrics.get('recall', 0):.3f} | "
          f"F1: {test_metrics.get('f1', 0):.3f} | "
          f"PR-AUC: {test_metrics.get('pr_auc', 0):.3f}")
    print(f"Training Time: {duration:.1f} sec")
    if max_gpu_mem:
        print(f"Peak GPU Memory: {max_gpu_mem:.1f} MB")
    print(f"Final RAM Usage: {final_ram:.1f} MB")

    return test_metrics.get('recall', 0), results

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

# Адаптивные размерности эмбеддингов: √(уникальных значений) + небольшой запас
cat_emb_dims = [
    min(16, max(4, int(np.sqrt(dim)) + 2))
    for dim in data.cat_dims
]

print(f"Размерности эмбеддингов: {cat_emb_dims}")
print(f"Суммарная размерность после конкатенации: {sum(cat_emb_dims) + data.num_numerical}")

# Инициализация модели
graphsage = GraphSAGE(
    cat_dims=data.cat_dims,
    cat_emb_dims=cat_emb_dims,
    num_numerical=data.num_numerical,
    dim_h=64,
    dim_out=2
).to(device)

# Обучение (функция train_with_monitoring без изменений)
recall_sage, results_sage = train_with_monitoring(
    graphsage, data, epochs=200, model_name="GraphSAGE (с эмбеддингами)"
)

Размерности эмбеддингов: [4, 5, 4, 4, 5]
Суммарная размерность после конкатенации: 34

Training GraphSAGE (с эмбеддингами) with resource monitoring
Epoch   0 | TL: 0.696 | TF1: 0.303 | VL: 0.696 | VF1: 0.303 | VRec: 0.603 | VPR: 0.206
Epoch  10 | TL: 0.674 | TF1: 0.393 | VL: 0.675 | VF1: 0.392 | VRec: 0.529 | VPR: 0.326
Epoch  20 | TL: 0.641 | TF1: 0.414 | VL: 0.643 | VF1: 0.412 | VRec: 0.667 | VPR: 0.340
Epoch  30 | TL: 0.635 | TF1: 0.412 | VL: 0.636 | VF1: 0.410 | VRec: 0.734 | VPR: 0.344
Epoch  40 | TL: 0.632 | TF1: 0.415 | VL: 0.633 | VF1: 0.414 | VRec: 0.689 | VPR: 0.348
Epoch  50 | TL: 0.631 | TF1: 0.417 | VL: 0.632 | VF1: 0.415 | VRec: 0.681 | VPR: 0.350
Epoch  60 | TL: 0.630 | TF1: 0.418 | VL: 0.630 | VF1: 0.417 | VRec: 0.688 | VPR: 0.352
Epoch  70 | TL: 0.628 | TF1: 0.420 | VL: 0.628 | VF1: 0.419 | VRec: 0.685 | VPR: 0.355
Epoch  80 | TL: 0.626 | TF1: 0.422 | VL: 0.627 | VF1: 0.420 | VRec: 0.671 | VPR: 0.358
Epoch  90 | TL: 0.628 | TF1: 0.423 | VL: 0.628 | VF1: 0.422 | VRec: 0

In [ ]:
# Размерности эмбеддингов (адаптивная эвристика)
cat_emb_dims = [min(16, max(4, int(np.sqrt(dim)) + 2)) for dim in data.cat_dims]

# Инициализация модели
gcn = GCN(
    cat_dims=data.cat_dims,
    cat_emb_dims=cat_emb_dims,
    num_numerical=data.num_numerical,
    dim_h=64,
    dim_out=2
).to(device)

# Обучение
recall_gcn, results_gcn = train_with_monitoring(
    gcn, data, epochs=80, model_name="GCN (с эмбеддингами)"
)


Training GCN (с эмбеддингами) with resource monitoring
